# <center>[Toxic Comment Classification Challenge](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge)<br>Capstone 3<br></center>
### <center>By Hector Cadeaux</center><br><br>
## Imports